# 🖼️ Week2: GPT Image 강의 노트 (Colab)


> **주의:** Colab 런타임을 재시작하면 `OPENAI_API_KEY` 환경 변수를 다시 설정해야 하니, 실습 전 반드시 키를 재입력하세요.


## 진행 순서
1. Quickstart: 환경 준비 및 기본 이미지 생성
2. GPT Image 개요 및 API 비교
3. Responses API로 이미지 생성 실습
4. 이미지 출력 옵션(사이즈 · 품질 · 배경)
5. 멀티턴 이미지 개선과 스트리밍
6. 이미지 편집 · 입력 이미지 활용
7. 이미지 분석(비전) 기본 예시
8. 비용 · 한계 · 참고 자료


## 1) Quickstart: 환경 준비 및 기본 이미지 생성


Responses API와 Image API를 사용하려면 SDK 설치와 API 키 등록이 먼저입니다. Colab 학생들이 헷갈리지 않도록 아래 셀을 순서대로 데모하세요.


In [ ]:
%pip install openai pillow


In [ ]:
import os
from getpass import getpass

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 키 설정 완료")


In [ ]:
from openai import OpenAI

client = OpenAI()
print("✅ 클라이언트 생성 완료")


## 2) GPT Image 개요 및 API 비교
- `gpt-image-1`은 텍스트·이미지를 모두 이해하고 생성하는 멀티모달 모델입니다.
- 이미지 생성은 **Responses API**와 **Image API** 두 가지 진입점을 제공하므로, 사용 사례에 따라 선택하세요.



| API | 주요 기능 | 추천 상황 |
| --- | --- | --- |
| Responses API | 이미지 생성/수정 도구를 대화 흐름에 통합, `previous_response_id`로 멀티턴 지원 | 챗봇, 단계별 수정, 하나의 맥락에서 이미지·텍스트를 함께 다뤄야 할 때 |
| Image API | 생성/수정/변형(variation) 전용 엔드포인트, 간단한 REST 호출 | 단일 이미지 생성, 서버 사이드 파이프라인, DALL·E 호환 기능 활용 |

> **모델 선택 팁**: 기본 실습은 `gpt-image-1`을 사용하되, 비용·지연을 고려해 `gpt-4.1` 계열을 함께 소개하세요.


## 3) Responses API로 이미지 생성 실습
Responses API에서 이미지 생성 도구를 호출하면 텍스트 생성과 동일한 흐름 안에서 결과를 받을 수 있습니다.


In [ ]:
import base64

prompt = "Generate an image of gray tabby cat hugging an otter with an orange scarf"

response = client.responses.create(
    model="gpt-5",
    input=prompt,
    tools=[{"type": "image_generation"}],
)

image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]

if image_data:
    image_bytes = base64.b64decode(image_data[0])
    with open("cat_and_otter.png", "wb") as f:
        f.write(image_bytes)
    print("✅ 이미지 저장 완료 -> cat_and_otter.png")
else:
    print("이미지 생성 결과가 존재하지 않습니다.")


> **해설 포인트**
> - `response.output`에서 `type == "image_generation_call"`인 항목을 찾아 base64 이미지를 추출합니다.
> - Responses API는 JSON 응답 하나에 텍스트/이미지/오류 이벤트가 섞여 있으므로 필터링 과정이 필수입니다.



## 4) 이미지 출력 옵션(사이즈 · 품질 · 배경)
- `size`: `1024x1024`, `1024x1536`, `1536x1024` 등 원하는 비율 선택
- `quality`: `low`, `medium`, `high` (품질이 높을수록 토큰 비용과 지연이 증가)
- `background`: `transparent`로 설정하면 PNG/WebP에서 투명 배경 가능
- `output_format`, `output_compression`으로 포맷 및 압축률 제어 가능



In [ ]:
styled_prompt = "Draw a 2D pixel art style sprite sheet of a tabby gray cat"

response = client.responses.create(
    model="gpt-5",
    input=styled_prompt,
    tools=[
        {
            "type": "image_generation",
            "size": "1024x1024",
            "quality": "high",
            "background": "transparent",
        }
    ],
)

image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]

if image_data:
    sprite_bytes = base64.b64decode(image_data[0])
    with open("sprite.png", "wb") as f:
        f.write(sprite_bytes)
    print("✅ 투명 배경 스프라이트 저장 완료 -> sprite.png")


## 5) 멀티턴 이미지 개선과 스트리밍
이미지 생성 후 `previous_response_id`를 사용하면 같은 대화 맥락에서 후속 지시를 내려 변형할 수 있습니다. 스트리밍 옵션을 켜면 생성 중간 결과도 살펴볼 수 있습니다.


In [ ]:
base_prompt = "Generate an image of gray tabby cat hugging an otter with an orange scarf"
response = client.responses.create(
    model="gpt-5",
    input=base_prompt,
    tools=[{"type": "image_generation"}],
)

image_calls = [
    output
    for output in response.output
    if output.type == "image_generation_call"
]

if not image_calls:
    raise RuntimeError("초기 이미지가 생성되지 않았습니다.")

# 후속 지시
response_followup = client.responses.create(
    model="gpt-5",
    previous_response_id=response.id,
    input="Make it look realistic, add soft lighting",
    tools=[{"type": "image_generation"}],
)

followup_data = [
    output.result
    for output in response_followup.output
    if output.type == "image_generation_call"
]

if followup_data:
    refined_bytes = base64.b64decode(followup_data[0])
    with open("cat_and_otter_realistic.png", "wb") as f:
        f.write(refined_bytes)
    print("✅ 후속 이미지 저장 -> cat_and_otter_realistic.png")


> **스트리밍 데모 팁**
> ```python
> stream = client.responses.create(
>     model="gpt-5",
>     input="Draw a river made of white owl feathers",
>     stream=True,
>     tools=[{"type": "image_generation", "partial_images": 2}],
> )
> for event in stream:
>     if event.type == "response.image_generation_call.partial_image":
>         # event.partial_image_b64를 파일로 저장하여 중간 과정을 확인
>         ...
> ```
> `partial_images` 값을 조정하면 중간 이미지를 몇 개까지 받을지 정의할 수 있습니다.


## 6) 이미지 편집 · 입력 이미지 활용
이미지 생성뿐 아니라 기존 이미지를 참고하거나 부분적으로 수정하는 기능도 데모할 수 있습니다.


In [ ]:
from pathlib import Path

def create_file(file_path: str) -> str:
    with open(file_path, "rb") as file_content:
        result = client.files.create(file=file_content, purpose="vision")
    return result.id

prompt = (
    "Generate a photorealistic gift basket on a white background labeled 'Relax & Unwind' "
    "containing all the items in the reference pictures."
)

base64_image = base64.b64encode(Path("soap.png").read_bytes()).decode("utf-8")
file_id = create_file("incense-kit.png")

response = client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {
                    "type": "input_image",
                    "image_url": f"data:image/png;base64,{base64_image}",
                },
                {
                    "type": "input_image",
                    "file_id": file_id,
                },
            ],
        }
    ],
    tools=[{"type": "image_generation"}],
)

edit_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]

if edit_data:
    edited_bytes = base64.b64decode(edit_data[0])
    with open("gift_basket.png", "wb") as f:
        f.write(edited_bytes)
    print("✅ 참고 이미지를 사용한 편집 결과 저장 -> gift_basket.png")


> **마스킹 데모 요약**
> - 편집할 기본 이미지와 마스크 이미지는 동일한 크기/포맷이어야 하며 마스크에는 알파 채널이 필요합니다.
> - GPT Image의 마스킹은 엄밀한 픽셀 교체보다는 프롬프트 기반으로 동작하므로, 원하는 영역에 대해 충분한 설명을 제공하세요.



## 7) 이미지 분석(비전) 기본 예시
이미지를 입력으로 제공하면 장면 묘사, 텍스트 추출, 주요 요소 식별 등을 수행할 수 있습니다.


In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/1024px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

vision_response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": "Describe the scene in Korean"},
                {
                    "type": "input_image",
                    "image_url": image_url,
                    "detail": "high",
                },
            ],
        }
    ],
)

print(vision_response.output_text)


> **강조할 내용**
> - `detail` 파라미터는 `auto`/`low`/`high`를 지원하며, 고품질 분석이 필요할 때 `high`를 사용합니다.
> - 로컬 이미지를 다룰 때는 `data:image/png;base64,...` 형태로 인라인 전달하거나 Files API로 업로드한 `file_id`를 사용할 수 있습니다.



## 8) 비용 · 한계 · 참고 자료
- **비용 구조**: 입력 텍스트 토큰 + 입력 이미지 토큰 + 출력 이미지 토큰의 합 (품질과 해상도가 높을수록 비용 증가)
- **지연 시간**: 고품질/고해상도 생성은 수십 초 이상 걸릴 수 있으며, 스트리밍으로 중간 결과를 제공할 수 있음
- **한계 사례**: 작은 글자, 복잡한 공간 배치, 의료/전문 도메인, 회전된 이미지 등은 정확도가 낮을 수 있음
- **콘텐츠 정책**: 모든 프롬프트와 결과물은 사용 정책에 따라 필터링되며, `moderation` 파라미터(`auto`/`low`)로 완화 수준 조절 가능
- **참고 링크**: [Image generation 가이드](https://platform.openai.com/docs/guides/image-generation), [Images API reference](https://platform.openai.com/docs/api-reference/images), [Vision 가이드](https://platform.openai.com/docs/guides/images-vision)



> 💡 **강의 진행 팁**
> - 각 섹션마다 Week2 실습 노트(`01-GPT5-Image-Practice.ipynb`)의 TODO와 연결하여 데모 → 실습 순서로 진행하세요.
> - 고품질 결과가 필요하므로 사전에 샘플 이미지를 생성해두고, 강의 중에는 설정 변경에 따른 차이를 보여주면 효과적입니다.

